In [153]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cpu


In [145]:
# Sift through angles and ratings to create data and labels

data = pd.read_csv('cleaned.csv')
y = []
x = []
results = []

broken = False
for row in data.iterrows():
    dicted = row[1].to_dict()
    if broken:
        x.append(results)
        broken = False

    if str(dicted[' rating']) != 'nan':
        y.append(dicted[' rating'])
        broken = True
    else:
        stage = 0 if dicted['stage'] == 'Drive' else 1
        results.append([dicted[' knee_angle'], dicted[' hand_distance'], dicted[' elbow_angle'], dicted[' hip_angle'], stage])

In [154]:
#padded_x = pad_sequences(x, value=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print(x_train)
print(y_train)

[[[5.135626592467319, 14.0, 12.468728269955175, 71.16438792128584, 1], [4.398705354995591, 14.0, 26.28497098734098, 68.7105619193482, 1], [4.73341400955353, 15.0, 57.9605652501558, 67.78240573048168, 1], [5.07994815433165, 14.0, 58.22716682715366, 66.96317015061608, 0], [5.051075439764328, 13.0, 55.406855519336766, 66.38460366963011, 0], [7.393939022747667, 13.0, 52.96103715437126, 65.83908487667571, 1], [7.6992564183370815, 15.0, 33.04812486997528, 65.39487608204838, 1], [5.114066041868755, 14.0, 50.68114777160556, 65.63754698468783, 0], [4.204901878894332, 17.0, 88.67641078041859, 65.55604521958347, 0], [3.728186394895424, 20.0, 91.94365464725168, 65.90984684383916, 0], [4.27896864119747, 22.0, 108.74529176925428, 66.70222473918479, 1], [4.810231384828112, 24.0, 112.66872514445936, 67.16634582208245, 1], [5.527948327824857, 26.0, 121.7653025307572, 68.19859051364818, 0], [6.362277722030007, 27.0, 136.60300827983085, 68.77418562665959, 0], [7.392008644573121, 29.0, 155.46163904055632,

In [147]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64).to(device)

In [158]:
# Define hyperparameters

input_size = 5
hidden_size = 128
num_layers = 2
num_classes = 12
num_epochs = 5
lr = 0.001

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size,  num_classes)

    def forward(self, x):
        hidden_states = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out

In [150]:
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)


In [151]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [161]:
for epoch in range(num_epochs):
    model.train()

    outputs = model(x_train_tensor)
    print(outputs)
    loss = criterion(outputs, y_train_tensor)

    # Calculate accuracy

    _, predicted_labels = torch.max(outputs, 1)
    correct_pred = (predicted_labels == y_train_tensor).sum().item()
    acc = correct_pred // len(y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print(f'Epoch : {epoch}')
    # print(f'Accuracy {acc}')
    # print(f'Loss : {loss}')


tensor([[-0.9725,  2.4247, -0.8155, -0.7706,  2.4373, -0.7218, -0.8492, -0.6082,
         -0.9641, -0.6474, -0.7789,  1.5064],
        [-0.9725,  2.4247, -0.8155, -0.7706,  2.4373, -0.7218, -0.8492, -0.6082,
         -0.9641, -0.6474, -0.7789,  1.5064],
        [-0.9725,  2.4247, -0.8155, -0.7706,  2.4373, -0.7218, -0.8492, -0.6082,
         -0.9641, -0.6474, -0.7789,  1.5064],
        [-0.9725,  2.4247, -0.8155, -0.7706,  2.4373, -0.7218, -0.8492, -0.6082,
         -0.9641, -0.6474, -0.7789,  1.5064],
        [-0.9725,  2.4247, -0.8155, -0.7706,  2.4373, -0.7218, -0.8492, -0.6082,
         -0.9641, -0.6474, -0.7789,  1.5064]], grad_fn=<AddmmBackward0>)
tensor([[-0.9729,  2.4258, -0.8159, -0.7710,  2.4383, -0.7222, -0.8496, -0.6086,
         -0.9645, -0.6477, -0.7793,  1.5077],
        [-0.9729,  2.4258, -0.8159, -0.7710,  2.4383, -0.7222, -0.8496, -0.6086,
         -0.9645, -0.6477, -0.7793,  1.5077],
        [-0.9729,  2.4258, -0.8159, -0.7710,  2.4383, -0.7222, -0.8496, -0.6086,
   